# project background
  A contractor is asking where should he/she to start his/her business. To start the work, we should first know what kind of business is more popular in a given area. Based on that information, we could plot a map with top 5 business kinds in each area. 
  The assumption is that if a business is more popular, the possiblity of new business success is higher. 

# Data source and solution
  As I'm new to this part, I will take the data in this class of New York data https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json. Also try to re-use the similar algo in the class to clustering neighbourhoods.
  The we can provide a simple algo to indicate whether the proposed business is good not.

# Codes for the assignment

In [74]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
#!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!pip install -U scikit-learn
# import k-means from clustering stage
# use DBSCAN for auto clustering
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#!pip install folium=0.5.0 # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Requirement already up-to-date: scikit-learn in c:\users\qbai\appdata\local\programs\python\python37-32\lib\site-packages (0.24.1)


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
# download New York neighourhood data
#!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']

In [6]:
# convert to data frame
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [7]:
# get New York latitude and longitude for plotting purpose
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [8]:
# foursquared credential
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius = 500

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]
        if ('groups' in results):
            results = results['groups'][0]['items']
        else:
            continue
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
# something wrong with full data, taking manhaton as example
#neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
ny_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                           latitudes=neighborhoods['Latitude'],
                           longitudes=neighborhoods['Longitude']
                          )

# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

In [60]:
ny_onehot = ny_onehot.drop(columns=["ATM","Airport Terminal","Art Gallery","Art Museum","Bank","Bridge","Building","Bus Line","Bus Station","Bus Stop","Business Service","Check Cashing Service","Church","College Academic Building","College Arts Building","College Basketball Court","Community Center","Concert Hall","Construction & Landscaping","Duty-free Shop","Exhibit","Field","Fountain","Gas Station","High School","Hill","Historic Site","History Museum","Lake","Library","Lounge","Luggage Store","Martial Arts School","Medical Center","Memorial Site","Metro Station","Monument / Landmark","Multiplex","Museum","Music School","Non-Profit","Opera House","Park","Pedestrian Plaza","Performing Arts Venue","Platform","Playground","Pool","Post Office","Rest Area","River","Road","Scenic Lookout","School","Sculpture Garden","Skate Park","Soccer Field","Stadium","State / Provincial Park","Tech Startup","Tennis Stadium","Theme Park","Toll Plaza","Tourist Information Center","Track","Trail","Train","Train Station","Volleyball Court"])
ny_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Christmas Market,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Bookstore,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,Himalayan Restaurant,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Lingerie Store,Liquor Store,Locksmith,Malay Restaurant,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Gym,Outdoor Sculpture,Outdoors & Recreation,Outlet Mall,Outlet Store,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pier,Piercing Parlor,Pilates Studio,Pizza Place,Plaza,Poke Place,Polish Restaurant,Pool Hall,Print Shop,Professional & Other Places,Pub,Public 

In [12]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [61]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

#neighborhoods_venues_sorted.head()

In [46]:
#ny_grouped.to_csv("ny_grouped.csv")

In [77]:
# clustering similiar neighbourhoods
ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

# run DBSCAN clustering
#epsilon = 0.3
#minimumSamples = 7
#db = DBSCAN(eps=epsilon, min_samples=minimumSamples).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kclusters = len(set(db.labels_))

# run k-means clustering
kclusters = 7
db = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

db.labels_[0:10] 


array([5, 5, 5, 6, 6, 0, 0, 0, 2, 2])

In [79]:
# add clustering labels
neighborhoods_venues_sorted = neighborhoods_venues_sorted.drop(columns = ['Cluster Labels'])
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', db.labels_)

ny_merged = neighborhoods

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [80]:
ny_merged = ny_merged.dropna()
ny_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,2.0,Pharmacy,Donut Shop,Ice Cream Shop,Deli / Bodega,Caribbean Restaurant
1,Bronx,Co-op City,40.874294,-73.829939,2.0,Fast Food Restaurant,Pizza Place,Baseball Field,Bagel Shop,Salon / Barbershop
2,Bronx,Eastchester,40.887556,-73.827806,6.0,Caribbean Restaurant,Deli / Bodega,Diner,Fast Food Restaurant,Bowling Alley
3,Bronx,Fieldston,40.895437,-73.905643,0.0,Music Venue,Plaza,Filipino Restaurant,Ethiopian Restaurant,Event Service
4,Bronx,Riverdale,40.890834,-73.912585,0.0,Home Service,Gym,Plaza,Women's Store,Ethiopian Restaurant


## plot map with similar clusters
So that we could have view of popular similiarity of clusters

In [81]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [89]:
ny_merged.loc[ny_merged['Cluster Labels'] == 0, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Fieldston,Music Venue,Plaza,Filipino Restaurant,Ethiopian Restaurant,Event Service
4,Riverdale,Home Service,Gym,Plaza,Women's Store,Ethiopian Restaurant
12,City Island,Deli / Bodega,Seafood Restaurant,Thrift / Vintage Store,Ice Cream Shop,Jewelry Store
46,Bay Ridge,Spa,Italian Restaurant,Pizza Place,American Restaurant,Bar
49,Greenpoint,Coffee Shop,Pizza Place,Bar,Cocktail Bar,Grocery Store
52,Sheepshead Bay,Dessert Shop,Turkish Restaurant,Yoga Studio,Karaoke Bar,Café
58,Windsor Terrace,Grocery Store,Deli / Bodega,Plaza,Bookstore,Beer Store
59,Prospect Heights,Bar,Mexican Restaurant,Wine Shop,Thai Restaurant,Coffee Shop
61,Williamsburg,Bagel Shop,Pizza Place,Coffee Shop,Yoga Studio,Liquor Store
62,Bushwick,Bar,Mexican Restaurant,Deli / Bodega,Coffee Shop,Discount Store


# check whether select a given business in a given neighborhood is a good idea
To verify the result, one could select a neighborhood name and a target business idea.
Also will estimate the result.
If it's in the most popular business, good idea is shown.
Otherwise, it lists proposed business for the reason.

In [91]:
def verify_proposal(neighborhood, business):
    businesses = neighborhoods_venues_sorted[neighborhoods_venues_sorted['Neighborhood']==neighborhood]
    if business in list(businesses.iloc[0][2:]):
        print("It's a good idea to setup {} in {}".format(business, neighborhood))
    else:
        print("Try to setup \n{} \nin \n{}".format(list(businesses.iloc[0][2:]), neighborhood))
    

In [92]:
# testing soluiton
business = "Chinese restaturant"
neighborhood = "Battery Park City"
verify_proposal(neighborhood, business)

Try to setup 
['Coffee Shop', 'Hotel', 'Clothing Store', 'Gym', 'Plaza'] 
in 
Battery Park City


# Discussion
  In the above section, we verified our assumption based on existing data for NY neighboring for most popular businesses.
  One may argue that we choose most popular business may lead to more competition, should it be a good idea? 
  Yes, as the factors for a successful business is more than we know, so it's just a quick idea to verify. 
  For food, maybe it's a good reference. 
  For other kind of business, good practice should also consider other factors, e.g: population, target client and so forth.

# Conclusion
 Whether the proposal is good or not still needs to be verified for the real world.
 We should continue to tune.